# Region-Based Hierarchical Clustering Color Quantization (RHCCQ)

## Introduction

RHCCQ is an adaptive image compression system that prioritizes visual quality in important regions while aggressively compressing less significant areas. Unlike standard compression methods, RHCCQ uses ROI detection to identify critical image regions and applies hierarchical color clustering to reduce colors without losing important details.

## How It Works

1. **ROI Detection**: Identifies important regions using edge density analysis
2. **Region Segmentation**: Divides image into ROI (high quality) and non-ROI (low quality)
3. **SubRegion Segmentation**: Each ROI and non-ROI gets divided in sub-regions
4. **Hierarchical Clustering**: Groups similar colors while preserving important variations
5. **Matrix Encoding**: Stores compressed color indices and optimized color palettes
5. **File Output**: Creates .rhccq files with region information and compressed data

## Technical Advantages

- **Better Detail Preservation**: Hierarchical clustering reduces color banding artifacts
- **Adaptive Bit Allocation**: More bits allocated to important regions
- **Configurable Trade-offs**: Users balance quality vs. compression based on application needs
- **Novel Approach**: Combines ROI awareness with advanced color quantization


## Methodology
The following code shows the whole process of the compression by using a famous image for Image Processing

In [ ]:
# Helper Functions to plot

import matplotlib.pyplot as plt

def plot_border_stages(intensity_borders, binary_borders, 
                       binary_thin_bordersless, noiseless_binary_borders,
                       pre_connected, connected, figsize=(16, 10)):
    """
    Plot all border processing stages in a 2x3 grid.
    """
    # Create subplots
    fig, axes = plt.subplots(2, 3, figsize=figsize)
    
    # Flatten axes for easy indexing
    axes = axes.flatten()
    
    # Plot each image with appropriate colormap
    images = [
        (intensity_borders, "Intensity Borders", "gray"),
        (binary_borders, "Binary Borders", "gray"),
        (binary_thin_bordersless, "Thinned Borders", "gray"),
        (noiseless_binary_borders, "Noiseless Borders", "gray"),
        (pre_connected, "Pre-Connected", "gray"),
        (connected, "Final Connected", "gray")
    ]
    
    for ax, (img, title, cmap) in zip(axes, images):
        # Handle binary vs intensity images
        if img.dtype == bool or img.max() == 1:
            ax.imshow(img, cmap=cmap, vmin=0, vmax=1)
        else:
            ax.imshow(img, cmap=cmap)
        
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.axis('off')
        
        # Add image stats
        if img.dtype == bool:
            stats = f"True: {img.sum():,}\nFalse: {(~img).sum():,}"
        else:
            stats = f"Min: {img.min():.1f}\nMax: {img.max():.1f}\nMean: {img.mean():.3f}"
        
        # Add stats in top-left corner
        ax.text(0.02, 0.98, stats,
                transform=ax.transAxes,
                fontsize=9,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.suptitle("Border Processing Pipeline", fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()

import matplotlib.pyplot as plt
import numpy as np

def plot_region_processing_pipeline(denoised, border_mask, protected_image, 
                                   bridged_image, closed_regions, cleaned_image, 
                                   region_map, figsize=(18, 10)):
    """
    Plot the region processing pipeline in a 3x3 grid (with one empty spot).
    """
    # Create subplots - 3x3 grid
    fig, axes = plt.subplots(3, 3, figsize=figsize)
    
    # Images and their titles
    images = [
        denoised,
        border_mask,
        protected_image,
        bridged_image,
        closed_regions,
        cleaned_image,
        region_map
    ]
    
    titles = [
        "1. Denoised Borders",
        "2. Border Mask", 
        "3. Protected Image",
        "4. Bridged Gaps",
        "5. Morphologically Closed",
        "6. Cleaned Regions",
        "7. Final Region Map"
    ]
    
    descriptions = [
        "Noise removed from binary borders",
        "Mask of detected border pixels",
        "Original image with border protection",
        "Small gaps bridged for connectivity",
        "After morphological closing operations",
        "Small regions filtered out",
        "Final ROI (white) / non-ROI (black) map"
    ]
    
    # Plot each image
    for idx, (img, title, desc) in enumerate(zip(images, titles, descriptions)):
        row = idx // 3
        col = idx % 3
        
        ax = axes[row, col]
        
        # Choose colormap based on content
        if 'region map' in title.lower() or 'mask' in title.lower():
            # Binary/border images
            if img.dtype == bool:
                ax.imshow(img, cmap='gray_r', vmin=0, vmax=1)  # gray_r for white=borders
            else:
                ax.imshow(img, cmap='gray_r')
        elif 'image' in title.lower():
            # Color images
            if len(img.shape) == 3 and img.shape[2] == 3:
                ax.imshow(img)
            else:
                ax.imshow(img, cmap='gray')
        else:
            # Default to grayscale
            ax.imshow(img, cmap='gray')
        
        ax.set_title(title, fontsize=12, fontweight='bold', color='darkgreen')
        ax.axis('off')
        
        # Add statistics
        stats_text = f"{desc}\n\n"
        
        if img.dtype == bool or np.max(img) <= 1:
            # Binary image stats
            true_count = np.sum(img)
            total_pixels = img.size
            percentage = (true_count / total_pixels) * 100
            
            if 'region' in title.lower():
                stats_text += f"ROI: {true_count:,} px\n"
                stats_text += f"({percentage:.1f}% of image)"
            else:
                stats_text += f"Edges: {true_count:,} px\n"
                stats_text += f"({percentage:.2f}%)"
        elif len(img.shape) == 2:
            # Grayscale image stats
            stats_text += f"Range: [{img.min():.0f}, {img.max():.0f}]\n"
            stats_text += f"Mean: {img.mean():.1f}"
        else:
            # Color image stats
            stats_text += f"Shape: {img.shape}"
        
        # Add stats box
        ax.text(0.02, 0.98, stats_text,
                transform=ax.transAxes,
                fontsize=9,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    # Hide the last two empty subplots (8th and 9th)
    axes[2, 1].axis('off')
    axes[2, 2].axis('off')
    
    plt.suptitle("Region Processing Pipeline - From Borders to ROI Map", 
                 fontsize=16, fontweight='bold', y=0.95)
    plt.tight_layout()
    plt.show()


def plot_subregions(bbox_region,bbox_mask,subregions_segments,segment_boundaries):

    import matplotlib.pyplot as plt
    import numpy as np

    # Debug visualization
    print(f"\n=== DEBUG: SLIC/Watershed Segments ===")
    print(f"bbox_region shape: {bbox_region.shape}")
    print(f"bbox_mask shape: {bbox_mask.shape}")
    print(f"Segments shape: {subregions_segments.shape}")
    print(f"Unique segment IDs: {np.unique(subregions_segments)}")

    # Create a colormap for segments - SIMPLE FIX
    num_segments = len(np.unique(subregions_segments))
    colors = plt.colormaps['tab20'].resampled(num_segments)

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # 1. Original bbox region
    axes[0, 0].imshow(bbox_region)
    axes[0, 0].set_title('Original Region')
    axes[0, 0].axis('off')

    # 2. Mask overlay
    axes[0, 1].imshow(bbox_region)
    mask_overlay = np.zeros((*bbox_mask.shape, 4))
    mask_overlay[bbox_mask] = [1, 0, 0, 0.3]  # Red overlay
    mask_overlay[~bbox_mask] = [0, 0, 0, 0]  # Transparent
    axes[0, 1].imshow(mask_overlay)
    axes[0, 1].set_title('Region Mask (red)')
    axes[0, 1].axis('off')

    # 3. Segments visualization
    segments_display = np.zeros((*subregions_segments.shape, 3))
    for seg_id in np.unique(subregions_segments):
        if seg_id == 0:  # Background
            continue
        mask = subregions_segments == seg_id
        color = colors(seg_id % colors.N)[:3]  # RGB only
        segments_display[mask] = color

    axes[0, 2].imshow(segments_display)
    axes[0, 2].set_title(f'Segments ({num_segments-1} segments)')
    axes[0, 2].axis('off')

    # 4. Segments WITH mask boundary
    axes[1, 0].imshow(segments_display)
    # Draw mask boundary
    from skimage.segmentation import find_boundaries
    mask_boundary = find_boundaries(bbox_mask, mode='inner')
    y, x = np.where(mask_boundary)
    axes[1, 0].scatter(x, y, c='red', s=1, alpha=0.5)
    axes[1, 0].set_title('Segments + Mask Boundary')
    axes[1, 0].axis('off')

    # 5. Problematic segments (outside mask)
    problematic_mask = np.zeros_like(bbox_mask, dtype=bool)
    for seg_id in np.unique(subregions_segments):
        if seg_id == 0:
            continue
        segment_mask = subregions_segments == seg_id
        outside_mask = segment_mask & ~bbox_mask
        if np.any(outside_mask):
            problematic_mask = problematic_mask | outside_mask

    axes[1, 1].imshow(bbox_region)
    if np.any(problematic_mask):
        problematic_overlay = np.zeros((*problematic_mask.shape, 4))
        problematic_overlay[problematic_mask] = [1, 0, 0, 0.7]  # Bright red
        axes[1, 1].imshow(problematic_overlay)
        axes[1, 1].set_title(f'Problem: {np.sum(problematic_mask)} pixels outside mask')
    else:
        axes[1, 1].set_title('Good: No pixels outside mask')
    axes[1, 1].axis('off')

    # 6. Boundaries visualization
    axes[1, 2].imshow(bbox_region)
    for boundary_info in segment_boundaries:
        seg_id = boundary_info['segment_id']
        boundary_coords = boundary_info['boundary_coords']
        
        if len(boundary_coords) < 3:
            continue
        
        # Convert to numpy array for plotting
        boundary_array = np.array(boundary_coords)
        
        # Get color for this segment
        color = colors(seg_id % colors.N)[:3]
        
        # Plot boundary
        axes[1, 2].plot(boundary_array[:, 1], boundary_array[:, 0], 
                    color=color, linewidth=1.5, alpha=0.8)
        
        # Label segment ID
        if len(boundary_array) > 0:
            centroid = np.mean(boundary_array, axis=0)
            axes[1, 2].text(centroid[1], centroid[0], str(seg_id),
                        color='white', fontsize=8, fontweight='bold',
                        ha='center', va='center',
                        bbox=dict(boxstyle='round,pad=0.2', 
                                    facecolor=color, alpha=0.7))

    axes[1, 2].set_title(f'Boundaries ({len(segment_boundaries)} segments)')
    axes[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

    # Print statistics
    print(f"\n=== SEGMENT STATISTICS ===")
    print(f"Total segments: {num_segments - 1} (excluding background 0)")

    problematic_segments = []
    for seg_id in np.unique(subregions_segments):
        if seg_id == 0:
            continue
        
        segment_mask = subregions_segments == seg_id
        total_pixels = np.sum(segment_mask)
        inside_pixels = np.sum(segment_mask & bbox_mask)
        outside_pixels = np.sum(segment_mask & ~bbox_mask)
        
        if outside_pixels > 0:
            problematic_segments.append(seg_id)
            print(f"Segment {seg_id}: {inside_pixels} inside, {outside_pixels} outside ({outside_pixels/total_pixels*100:.1f}% outside)")
        else:
            print(f"Segment {seg_id}: {inside_pixels} inside, fully within mask")

    if problematic_segments:
        print(f"\n⚠️  PROBLEM: {len(problematic_segments)} segments extend outside mask!")
    else:
        print(f"\n✓ GOOD: All segments are within mask")

    # Also check boundary extraction
    print(f"\n=== BOUNDARY EXTRACTION ===")
    print(f"Extracted boundaries for {len(segment_boundaries)} segments")
    for i, boundary_info in enumerate(segment_boundaries[:5]):  # Show first 5
        print(f"  Segment {boundary_info['segment_id']}: {boundary_info['num_points']} boundary points, area={boundary_info['area']}")

    

def show_reconstruction(seg_compression, seg_idx,segment_image_cropped,top_left_abs_row,top_left_abs_col):
        
        from decoder.uncompression.uncompression import partial_decompress_color_quantization

        reconstruction_result = partial_decompress_color_quantization(
            seg_compression,
        )


        # ==============================================
        # 4. VISUALIZE COMPARISON WITH STATS
        # ==============================================
        print(f"\n{'='*60}")
        print(f"SEGMENT {seg_idx+1} - COLOR QUANTIZATION RESULTS")
        print(f"{'='*60}")

        # Get stats from compression
        h, w, _ = segment_image_cropped.shape
        original_size = h * w * 3
        compressed_size = seg_compression['compressed_size']
        compression_ratio = seg_compression['compression_ratio']
        psnr = seg_compression['psnr']
        n_colors = seg_compression['compressed_colors']

        # Print stats in a clean format
        print(f"Segment {seg_idx+1} Summary:")
        print(f"  Shape: {h}x{w}")
        print(f"  Top-left: {top_left_abs_row, top_left_abs_col}")
        print(f"  Colors in palette: {n_colors}")
        print(f"  Original size: {original_size:,} bytes")
        print(f"  Compressed size: {compressed_size:,} bytes")
        print(f"  Compression ratio: {compression_ratio:.2f}:1")
        print(f"  Space savings: {(1 - compressed_size/original_size)*100:.1f}%")
        print(f"  Quality (PSNR): {psnr:.2f} dB")

        # Create visualization figure
        import matplotlib.pyplot as plt

        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        # Original image
        axes[0, 0].imshow(segment_image_cropped)
        axes[0, 0].set_title(f'Original\n{h}x{w} pixels')
        axes[0, 0].axis('off')

        # Add pixel info
        non_black = np.sum(np.any(segment_image_cropped > 10, axis=2))
        axes[0, 0].text(0.02, 0.98, f'Content: {non_black:,} px', 
                    transform=axes[0, 0].transAxes, fontsize=9, color='white',
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

        # Reconstructed image
        reconstructed_img = reconstruction_result['image']
        axes[0, 1].imshow(reconstructed_img)
        axes[0, 1].set_title(f'Reconstructed\n{n_colors} colors')
        axes[0, 1].axis('off')

        # Difference (amplified for visibility)
        diff = np.abs(segment_image_cropped.astype(float) - reconstructed_img.astype(float))
        diff_display = np.clip(diff * 3, 0, 255).astype(np.uint8)
        axes[0, 2].imshow(diff_display)
        axes[0, 2].set_title(f'Difference (×3)\nPSNR: {psnr:.2f} dB')
        axes[0, 2].axis('off')

        # Add MSE/PSNR to difference plot
        mse = np.mean(diff ** 2)
        axes[0, 2].text(0.02, 0.98, f'MSE: {mse:.2f}', 
                    transform=axes[0, 2].transAxes, fontsize=9, color='white',
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='red', alpha=0.7))

        # Color palette visualization
        palette = np.array(seg_compression['palette'])
        axes[1, 0].axis('off')

        # Create palette visualization
        if n_colors > 0:
            # Create a color swatch
            palette_h = max(1, min(10, n_colors // 10))
            palette_w = (n_colors + palette_h - 1) // palette_h
            
            palette_img = np.zeros((palette_h * 20, palette_w * 20, 3), dtype=np.uint8)
            
            for idx, color in enumerate(palette):
                row = (idx // palette_w) * 20
                col = (idx % palette_w) * 20
                palette_img[row:row+18, col:col+18] = color
            
            # Display in inset axes
            from mpl_toolkits.axes_grid1.inset_locator import inset_axes
            ax_inset = inset_axes(axes[1, 0], width="60%", height="60%", loc='center')
            ax_inset.imshow(palette_img)
            ax_inset.set_title(f'Color Palette ({n_colors} colors)')
            ax_inset.axis('off')
            
            # Add palette stats
            unique_original = len(np.unique(segment_image_cropped.reshape(-1, 3), axis=0))
            palette_text = f'Original: {unique_original} colors\nCompressed: {n_colors} colors\nReduction: {(1 - n_colors/unique_original)*100:.1f}%'
            axes[1, 0].text(0.5, 0.1, palette_text, transform=axes[1, 0].transAxes,
                        fontsize=9, horizontalalignment='center',
                        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

        # Compression statistics visualization
        axes[1, 1].axis('off')

        # Create bar chart for size comparison
        size_data = [original_size, compressed_size]
        size_labels = ['Original', 'Compressed']
        colors = ['skyblue', 'lightcoral']

        # Create inset axes for bar chart
        ax_bar = inset_axes(axes[1, 1], width="60%", height="60%", loc='center')
        bars = ax_bar.bar(size_labels, size_data, color=colors, alpha=0.7)
        ax_bar.set_title('Size Comparison')
        ax_bar.set_ylabel('Bytes')

        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax_bar.text(bar.get_x() + bar.get_width()/2., height + max(size_data)*0.05,
                    f'{height:,}', ha='center', va='bottom', fontsize=9)

        # Add ratio text
        ratio_text = f'Compression Ratio: {compression_ratio:.2f}:1\nSavings: {(1 - compressed_size/original_size)*100:.1f}%'
        axes[1, 1].text(0.5, 0.1, ratio_text, transform=axes[1, 1].transAxes,
                    fontsize=9, horizontalalignment='center',
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

        # Index map visualization (shows which palette index each pixel uses)
        axes[1, 2].axis('off')

        # Get index map
        index_map = np.array(seg_compression['indices']).reshape(h, w)

        # Display index map as grayscale
        ax_index = inset_axes(axes[1, 2], width="60%", height="60%", loc='center')
        im = ax_index.imshow(index_map, cmap='viridis', aspect='auto')
        ax_index.set_title('Index Map (Palette Indices)')
        ax_index.axis('off')

        # Add colorbar for index map
        plt.colorbar(im, ax=ax_index, fraction=0.046, pad=0.04)

        # Add index map stats
        index_unique = len(np.unique(index_map))
        axes[1, 2].text(0.5, 0.1, f'Unique indices: {index_unique}/{n_colors}',
                    transform=axes[1, 2].transAxes,
                    fontsize=9, horizontalalignment='center',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

        # Main title
        plt.suptitle(f'Segment {seg_idx+1}: Color Quantization Results\nTop-left: {top_left_abs_row, top_left_abs_col}, Size: {h}x{w}', 
                    fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # ==============================================
        # 5. CONSOLE SUMMARY (CLEAN FORMAT)
        # ==============================================
        print(f"\n📊 COMPRESSION SUMMARY:")
        print(f"   Original:    {original_size:>8,} bytes")
        print(f"   Compressed:  {compressed_size:>8,} bytes")
        print(f"   Ratio:       {compression_ratio:>8.2f}:1")
        print(f"   Savings:     {(1 - compressed_size/original_size)*100:>7.1f}%")
        print(f"   PSNR:        {psnr:>8.2f} dB")
        print(f"   Palette:     {n_colors:>8} colors")
        print(f"{'='*60}")


### Loading of the Image

In [ ]:
import cv2
from encoder.enhancer.clahe import get_enhanced_image

image_name = "images/png/Lenna.png"
image = cv2.imread(image_name)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


import matplotlib.pyplot as plt

# Display the image
plt.figure(figsize=(8, 6))  # Optional: Set figure size
plt.imshow(image_rgb)
plt.axis('off')  # Hide axes
plt.title("Lenna Image (RGB)")
plt.show()

### Region of Interest Detection

In [ ]:

import math
import numpy as np
from encoder.ROI.edges import compute_local_density, suggest_automatic_threshold, get_edge_map
from encoder.ROI.thin_regions2 import remove_thin_structures_optimized
from encoder.ROI.small_regions import remove_small_regions, connect_nearby_pixels, connect_by_closing_fast
from encoder.ROI.small_gaps import bridge_small_gaps, bridge_small_gaps_fast

from encoder.ROI.roi import remove_small_noise_regions, detect_meaningful_borders, protect_border_regions, fill_closed_regions, extract_roi_nonroi,visualize_roi_nonroi_comparison


import matplotlib.pyplot as plt

roi_quality=20
nonroi_quality=10


# Get edge and density maps
edge_map = get_edge_map(image_rgb)
edge_density = compute_local_density(edge_map, kernel_size=3)

plt.imshow(edge_map)
plt.show()

plt.imshow(edge_density)
plt.show()

# Variables used for ROI Detection
density_threshold = suggest_automatic_threshold(edge_density, edge_map, method="mean") / 100
min_region_size= math.ceil( image_rgb.size / math.pow(10, math.ceil(math.log(image_rgb.size, 10))-3 ) ) 



"""
Complete pipeline: filter edges by density, then unify regions, remove small regions.
Returns ROI and non-ROI separately.
"""

# Get high-density borders
high_density_mask = edge_density > density_threshold
intensity_borders = edge_map.copy()
intensity_borders[~high_density_mask] = 0

# Convert to binary (0 and 255)
binary_borders = (intensity_borders > 0).astype(np.uint8) * 255

# Remove tgin structures and noise from the picture
binary_thin_bordersless=remove_thin_structures_optimized(binary_borders, density_threshold=0.10, thinness_threshold=0.3, window_size=25, min_region_size=25) 
noiseless_binary_borders=remove_small_noise_regions(binary_thin_bordersless, min_size=75)

# Connect Regions Close to each other
pre_connected = connect_by_closing_fast(
    noiseless_binary_borders,
    connection_distance=5,
    min_region_size=25
)
connected = bridge_small_gaps_fast(pre_connected, max_gap=100, density_threshold=0.2, local_window=15, regional_window=25)


plot_border_stages(intensity_borders, binary_borders, 
                   binary_thin_bordersless, noiseless_binary_borders,
                   pre_connected, connected) 



"""
Unify regions while respecting natural borders and directional context.
"""

# Ensure binary image is 0-255 uint8
#if connected.max() <= 1:
binary_image = (connected * 255).astype(np.uint8)

denoised=binary_image

# Detect strong borders using gradient
border_mask = detect_meaningful_borders(denoised, sensitivity=0.5)

# Protect borders from being unified
protected_image = protect_border_regions(denoised, border_mask, kernel_size=15)

# Bridge small gaps within regions (not across borders), fill small closed regions and remove the remainings
bridged_image = bridge_small_gaps_fast(protected_image, max_gap=25, density_threshold=0.2, local_window=15, regional_window=25)
closed_regions=fill_closed_regions(bridged_image, min_hole_size=10, max_hole_size=10000, connectivity=4)
cleaned_image = remove_small_regions(closed_regions, min_size=5, remove_thin_lines=True, kernel_size=30)

# Create region map
region_map = (cleaned_image > 0).astype(np.uint8)


plot_region_processing_pipeline(
    denoised, 
    border_mask, 
    protected_image, 
    bridged_image, 
    closed_regions, 
    cleaned_image, 
    region_map
)

# Extract ROI and non-ROI
roi_image, nonroi_image, roi_mask, nonroi_mask = extract_roi_nonroi(image_rgb, region_map)

visualize_roi_nonroi_comparison(image_rgb, roi_image, nonroi_image, region_map)


In [ ]:
from encoder.ROI.roi import  extract_regions

roi_regions, nonroi_regions = extract_regions(image_rgb, roi_mask, nonroi_mask)

print(f"Found {len(roi_regions)} ROI regions")
print(f"Found {len(nonroi_regions)} non-ROI regions")

# Display some statistics
print("\nROI Regions (sorted by area):")
for i, region in enumerate(sorted(roi_regions, key=lambda x: x['area'], reverse=True)[:5]):
    print(f"  Region {i+1}: Area = {region['area']} pixels")

print("\nNon-ROI Regions (sorted by area):")
for i, region in enumerate(sorted(nonroi_regions, key=lambda x: x['area'], reverse=True)[:5]):
    print(f"  Region {i+1}: Area = {region['area']} pixels")

### SubRegion Quantization
In this phase we have the SLIC function applied to every region in order to find SubRegions. For each SubRegions we find its unique colors and we perform teh clustering based off the quality in order to reduce the colors.

In [ ]:
from encoder.subregions.split_score import calculate_split_score, normalize_result
from encoder.subregions.slic import visualize_split_analysis, enhanced_slic_with_texture, extract_slic_segment_boundaries
from encoder.compression.clustering import compute_clustering_params,cluster_palette_colors_parallel, get_all_unique_colors
from encoder.compression.merging import merge_region_components_simple, visualize_merged_result


subregions_components=[]

for i, region in enumerate(roi_regions):

    region_components=[]
    
    # ==============================================
    # 1. EXTRACT THE subregions
    # ==============================================
    minr, minc, maxr, maxc = region['bbox']
    bbox_region = image_rgb[minr:maxr, minc:maxc]
    bbox_mask = region['bbox_mask']  # Irregular region mask
    region_image = bbox_region
    
    print(f"Region {i+1}: {bbox_region.shape[1]}x{bbox_region.shape[0]} pixels")
    print(f"Mask area: {np.sum(bbox_mask):,} pixels")

    
    # ==============================================
    # 2. ANALYZE REGION FOR SEGMENTATION
    # ==============================================
    overall_score, color_score, texture_score = calculate_split_score(bbox_region, bbox_mask)
    
    print(f"  Analysis scores:")
    print(f"    Overall: {overall_score:.3f}")
    print(f"    Color: {color_score:.3f}")
    print(f"    Texture: {texture_score:.3f}")
    
    window = math.ceil(math.ceil(math.log(bbox_region.size, 10)) * math.log(bbox_region.size))
    normalized_overall_score = normalize_result(overall_score, window)
    optimal_segments = math.ceil(normalized_overall_score)
    
    if optimal_segments <= 0: 
        optimal_segments = 1
    
    
    print(f"  Optimal segments: {optimal_segments}")
    
    # ==============================================
    # 3. APPLY SLIC SEGMENTATION TO THE subregions
    # ==============================================
    if optimal_segments<1: optimal_segments=1
    subregions_segments, texture_map = enhanced_slic_with_texture(bbox_region, bbox_mask, n_segments=optimal_segments)
    segment_boundaries = extract_slic_segment_boundaries(subregions_segments, bbox_mask)

    print(f"  Found {len(segment_boundaries)} sub-regions")
    print(f"  Total boundary points: {sum(seg['num_points'] for seg in segment_boundaries):,}")

    plot_subregions(bbox_region,bbox_mask,subregions_segments,segment_boundaries)













    for seg_idx, seg_data in enumerate(segment_boundaries):
        segment_id = seg_data.get('segment_id', seg_idx)
        segment_mask = (subregions_segments == segment_id) & bbox_mask
        
        import numpy as np
        segment_pixels = np.sum(segment_mask)
        
        print(f"\n    Segment {seg_idx+1}/{len(segment_boundaries)} (ID: {segment_id}):")
        print(f"      Pixels: {segment_pixels:,}")
        
        # ==============================================
        # 1. CREATE CORRECT SEGMENT IMAGE
        # ==============================================
        
        # Get the ORIGINAL pixels for this segment only
        segment_pixels_original = region_image[segment_mask]
        
        # ==============================================
        # 2. FIND TIGHT BOUNDING BOX FROM SEGMENT MASK
        # ==============================================
        # Use the SEGMENT MASK, not the modified image!
        rows, cols = np.where(segment_mask)
        
        if len(rows) == 0 or len(cols) == 0:
            continue
        
        # Find bounding box coordinates FROM THE MASK
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()
        
        # Add small padding
        pad = 2
        h, w = region_image.shape[:2]
        min_row = max(0, min_row - pad) 
        max_row = min(h - 1, max_row + pad) 
        min_col = max(0, min_col - pad)
        max_col = min(w - 1, max_col + pad)
        
        # Calculate dimensions
        crop_height = max_row - min_row + 1
        crop_width = max_col - min_col + 1
        
        # ==============================================
        # 3. CROP ORIGINAL IMAGE TO BBOX
        # ==============================================
        # Crop the ORIGINAL region image (not modified)
        bbox_crop = region_image[min_row:max_row+1, min_col:max_col+1]
        
        # Also crop the segment mask to same bbox
        segment_mask_cropped = segment_mask[min_row:max_row+1, min_col:max_col+1]
        
        # Create final segment image: only segment pixels visible
        segment_image_cropped = np.zeros_like(bbox_crop)
        segment_image_cropped[segment_mask_cropped] = bbox_crop[segment_mask_cropped]
        
        # ==============================================
        # 4. CALCULATE ABSOLUTE COORDINATES
        # ==============================================            
        absolute_min_row = min_row + minr
        absolute_min_col = min_col + minc 
        absolute_max_row = max_row + minr 
        absolute_max_col = max_col + minc
        
        top_left_abs_row = absolute_min_row
        top_left_abs_col = absolute_min_col
        
        print(f"      Original region shape: {region_image.shape[:2]}")
        print(f"      Cropped bbox shape: {segment_image_cropped.shape[:2]}")
        print(f"      Segment pixels in crop: {np.sum(segment_mask_cropped)}")
        print(f"      Top-left in full image: ({top_left_abs_row}, {top_left_abs_col})")


        # Get segment pixels
        segment_pixels = bbox_crop[segment_mask_cropped]

        if len(segment_pixels) > 0:
            # Check for black pixels INSIDE the segment (not background)
            black_in_segment = np.any(np.all(segment_pixels == [0, 0, 0], axis=1))
            
            if black_in_segment:
                print(f"Segment {segment_id}: Has black pixels inside segment")
                
                # Replace black pixels with nearest non-black color
                # First, find non-black pixels
                non_black_mask = ~np.all(segment_pixels == [0, 0, 0], axis=1)
                non_black_pixels = segment_pixels[non_black_mask]
                
                if len(non_black_pixels) > 0:
                    # For each black pixel, find closest non-black color
                    black_mask = np.all(segment_pixels == [0, 0, 0], axis=1)
                    black_indices = np.where(black_mask)[0]
                    
                    for idx in black_indices:
                        # Calculate distances to all non-black pixels
                        distances = np.linalg.norm(non_black_pixels - segment_pixels[idx], axis=1)
                        closest_idx = np.argmin(distances)
                        segment_pixels[idx] = non_black_pixels[closest_idx]
                    
                    print(f"  Fixed {len(black_indices)} black pixels in segment")
                
                # Update the segment image
                segment_image_cropped[segment_mask_cropped] = segment_pixels

        # ==============================================
        # 2. COMPRESS CROPPED SEGMENT
        # ==============================================
        seg_compression = get_all_unique_colors(
            segment_image_cropped,
            (top_left_abs_row, top_left_abs_col)
        )

        n_colors = seg_compression['actual_colors']

        palette_colors = np.array(seg_compression['palette'])
        
        # Option A: Simple improved formulas
        eps, min_samples, max_colors_per_cluster = compute_clustering_params(
            n_colors, roi_quality, color_space='lab'
        )

        

        # Then cluster the colors
        seg_compression = cluster_palette_colors_parallel(
            roi_quality,
            seg_compression,
            eps=eps,           # Distance threshold (0-255 scale)
            min_samples=1,      # Min colors to form cluster
            max_colors_per_cluster=max_colors_per_cluster  # Split large clusters
        )

        
        print(f"Eps: {eps}")
        print(f"n_colors: {n_colors}")
        print(f"max_colors_per_cluster: {max_colors_per_cluster}")

        
        #show_reconstruction(seg_compression, seg_idx,segment_image_cropped,top_left_abs_row,top_left_abs_col)

        region_components.append(seg_compression)
            
        
            

        #all_segments_compressed.append(seg_compression)
        region_components.append(seg_compression)


    # ==============================================
    # 4. MERGE COMPONENTS WITHIN THIS subregions
    # ==============================================
    if len(region_components) > 1:
        # Get subregions bbox
        minr, minc, maxr, maxc = region['bbox']
        subregions_bbox = (minr, minc, maxr, maxc)
        subregions_height = maxr - minr
        subregions_width = maxc - minc
        
        # Choose merging strategy
        # Simple merge (colored pixels override black)
        merged_components = merge_region_components_simple(region_components, subregions_bbox)
        subregions_components.append(merged_components)

        visualize_merged_result(merged_components, (subregions_height, subregions_width), minr, minc)
        
        # Calculate statistics
        original_pixels = sum(seg['shape'][0] * seg['shape'][1] for seg in region_components)
        original_black = sum(seg['indices'].count(1) for seg in region_components)
        
        merged_pixels = sum(seg['shape'][0] * seg['shape'][1] for seg in merged_components)
        merged_black = sum(seg['indices'].count(1) for seg in merged_components)
        
        print(f"\nSummary:")
        print(f"  Original: {len(region_components)} segments, {original_pixels:,} pixels")
        print(f"  Merged: {len(merged_components)} segments, {merged_pixels:,} pixels")
        print(f"  Black reduction: {original_black - merged_black:,} pixels")
        print(f"  Pixel reduction: {original_pixels - merged_pixels:,} pixels")
        
    else:
        # Just add the single component
        subregions_components.append(region_components)
        print(f"Only 1 component in subregions {i+1}, no merging needed")

In [ ]:
from encoder.compression.subregions import subregion_quantization
#Emulate teh behaviour for next computation
ROI_subregions_components=subregion_quantization(image_rgb, roi_regions, quality=roi_quality, subregion_type="ROI", debug=False)
nonROI_subregions_components=subregion_quantization(image_rgb, nonroi_regions, quality=nonroi_quality, subregion_type="nonROI", debug=False)

### Region Clustering
This is the second phase of the Hierarchical clustering. Clustering colors between each ROI / nonROI.
The same approach from before is used.

In [ ]:
from encoder.compression.regions import region_quantization

roi_region_quality=roi_quality*2
nonroi_region_quality=nonroi_quality*2

if roi_region_quality>100: roi_region_quality=100
if nonroi_region_quality>100: nonroi_region_quality=100

original_image_height, original_image_width, _ = image_rgb.shape

try: roi_components=region_quantization(ROI_subregions_components, quality=roi_region_quality, original_image_height=original_image_height, original_image_width=original_image_width)
except: roi_components=[]

try: nonroi_components=region_quantization(nonROI_subregions_components, quality=nonroi_region_quality, original_image_height=original_image_height, original_image_width=original_image_width)
except: nonroi_components=[]

image_components=roi_components+nonroi_components

### Image Quantization
Same process is applied to the last step of the clustering. Here we cluster the colors of all the regions.

In [ ]:
from encoder.compression.image import quantize_image

image_quality=roi_region_quality+nonroi_region_quality
if image_quality>100: image_quality=100

image_seg_compression=quantize_image(image_components, quality=image_quality, original_image_height=original_image_height, original_image_width=original_image_width)

### Saving the compression
In order to perform the compression, we compress the palette of colors and all the matrix of indeces

In [ ]:
from encoder.compression.compression import save_compressed, compress_palette, compress_indices_simple_optimized

filename="images/rhccq_20_10/kauai_compressed.rhccq"


def lossless_compress_optimized(palette, indices_list, shape, use_manual_rle=False):
    """
    Optimized compression with automatic dtype selection.
    Handles both Python lists and numpy arrays.
    """
    # Compress palette
    palette_compressed = compress_palette(palette)
    
    # Handle different input types
    if isinstance(indices_list, np.ndarray):
        # It's a numpy array
        if indices_list.size == 0:
            max_index = 0
            indices_flat = indices_list.flatten()
        else:
            max_index = indices_list.max()
            indices_flat = indices_list.flatten()
    elif isinstance(indices_list, list):
        # It's a Python list
        if not indices_list:
            max_index = 0
            indices_flat = indices_list
        else:
            max_index = max(indices_list)
            indices_flat = indices_list
    else:
        raise TypeError(f"indices_list must be list or numpy array, got {type(indices_list)}")
    
    # Determine optimal dtype for indices
    if max_index < 256:
        dtype = np.uint8
        dtype_name = 'uint8'
    elif max_index < 65536:
        dtype = np.uint16
        dtype_name = 'uint16'
    else:
        dtype = np.uint32
        dtype_name = 'uint32'
    
    print(f"Indices optimization: max={max_index}, using {dtype_name}")
    
    indices_compressed = compress_indices_simple_optimized(indices_flat, dtype)
    method = 'zlib_direct'
    
    return {
        's': shape,
        'l': len(palette),
        'p': palette_compressed,
        'i': indices_compressed,
        'd': dtype_name,  # Store dtype for decompression
        #'method': method
    }


# Get your final data
final_data = image_seg_compression  # or clustered_result

# Extract components
shape = final_data['shape']
palette = final_data['palette']
indices_flat = final_data['indices']

# Convert to matrix
h, w = shape
indices_matrix = np.array(indices_flat).reshape(h, w)

print(f"Compressing: {w}x{h} image, {len(palette)} colors")

# Compress
from encoder.compression.compression import save_compressed

compressed_data = lossless_compress_optimized(palette, indices_matrix, shape)

# Save
file_size = save_compressed(compressed_data, filename)

# Stats
original_size = h * w * 3
compression_ratio = original_size / file_size

print(f"✅ Saved: {filename}")
print(f"   Original: {original_size:,} bytes")
print(f"   Compressed: {file_size:,} bytes")
print(f"   Ratio: {compression_ratio:.2f}:1")
print(f"   Savings: {(1 - file_size/original_size)*100:.1f}%")

### Comparison of the Compressed picture with the original

In [ ]:
from decoder.uncompression.uncompression import lossless_decompress, load_compressed, decompress_color_quantization
from decoder.uncompression.comparison import calculate_quality_metrics, create_difference_visualization, print_quality_report, plot_comparison, calculate_adaptive_quality_metrics, print_adaptive_metrics

original=image_rgb

# Load and decompress
loaded = load_compressed(filename)
compressed=lossless_decompress(loaded)
palette_restored, indices_restored, shape_restored = lossless_decompress(loaded)

debug=False
if debug:
    print("\nDecompressed data:")
    print(f"Palette: {palette_restored}")
    print(f"Indices: {indices_restored}")
    print(f"Shape: {shape_restored}")


reconstruction_result = decompress_color_quantization(compressed)
reconstructed = reconstruction_result['image']  # This is a numpy array


import matplotlib.pyplot as plt
plt.title("reconstructed")
plt.imshow(reconstructed)
plt.show()


print(f"\nImage sizes:")
print(f"  Original: {original.shape[1]}x{original.shape[0]}")
print(f"  Reconstructed: {reconstructed.shape[1]}x{reconstructed.shape[0]}")

# Calculate metrics
print("\nCalculating quality metrics...")
metrics=calculate_adaptive_quality_metrics(original, reconstructed)

# Create difference visualizations
print("Creating difference visualizations...")
differences = create_difference_visualization(original, reconstructed)

# Print report
#print_quality_report(metrics)
print_adaptive_metrics(metrics, original.shape)


metrics = calculate_quality_metrics(original, reconstructed)
# Create comprehensive plot
print("\nGenerating comparison visualization...")
plot_comparison(original, reconstructed, metrics, differences)